In [1]:
# Image Style Transfer using convolutional neural networks using ResNet50
# code by GunhoChoi
# Referenced documents below
# https://discuss.pytorch.org/t/how-to-extract-features-of-an-image-from-a-trained-model/119/3
# https://github.com/leongatys/PytorchNeuralStyleTransfer

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import torch.utils.data as data
import torchvision.models as models
import torchvision.utils as v_utils
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [2]:
# content & style directory

content_dir = "./image/content/Neckarfront_origin.jpg"
style_dir = "./image/style/monet.jpg"

# hyperparameters

content_layer_num = 1
image_size = 256
epoch = 2500

In [3]:
# import pretrained resnet50 model
# check what layers are in the model
# if you want to see more specifically, print module

resnet = models.resnet50(pretrained=True) #ResNet50을 불러 온다.
for name,module in resnet.named_children(): #ResNet50의 구조
    print(name)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /Users/geunseong-gai/.torch/models/resnet50-19c8e357.pth
100.0%


conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [8]:
# resnet without fully connected layers
# return activations in each layers 

class Resnet(nn.Module):
    #불러온 pre-trained ResNet50은 우리가 사용하려는 구조와 조금 다르다. 따라서 이를 가져와 조금 변형한 모델을 직접 생성한다.
    def __init__(self):
        super(Resnet, self).__init__()
        self.layer0 = nn.Sequential(*list(resnet.children())[0:1]) #conv1
        self.layer1 = nn.Sequential(*list(resnet.children())[1:4]) #bn1, relu, maxpool
        self.layer2 = nn.Sequential(*list(resnet.children())[4:5]) #layer1
        self.layer3 = nn.Sequential(*list(resnet.children())[5:6]) #layer2
        self.layer4 = nn.Sequential(*list(resnet.children())[6:7]) #layer3
        self.layer5 = nn.Sequential(*list(resnet.children())[7:8]) #layer4
        #Resnet의 마지막 avgpool과 fc를 제거한다.

    def forward(self,x):
        out_0 = self.layer0(x)
        out_1 = self.layer1(out_0)
        out_2 = self.layer2(out_1)
        out_3 = self.layer3(out_2)
        out_4 = self.layer4(out_3)
        out_5 = self.layer5(out_4)

        return out_0, out_1, out_2, out_3, out_4, out_5 #각 레이어의 output을 각각 반환한다.

In [ ]:
# read & preprocess image
# normalize by subtracting imagenet mean of rgb channels

def image_preprocess(img_dir):
    img = Image.open(img_dir) #이미지를 불러온다.
    transform = transforms.Compose([
        transforms.Scale(image_size), #이미지 크기 변환
        transforms.CenterCrop(image_size), #중앙에 위치하도록 자른다.
        transforms.ToTensor(), #Torch Tensor로 변환한다. 0 ~ 1 사이 값을 가진다.
        transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961], 
                             std=[1,1,1]), #이미지 정규화
        #ImageNet : 평균 = [0.485, 0.456, 0.406] 및 표준편차 = [0.229, 0.224, 0.225]로 정규화 된 각 채널의 이미지에 대해 학습된 모델
    ])
    #transforms은 선처리를 위한 함수, transforms.Compose는 선처리를 위한 컨테이너이다.
    
    img = transform(img).view((-1,3,image_size,image_size)) #이미지를 선처리 해주고, size 변환한다.
    return img

# image post process
# add imagenet mean to normalize the output image
# bound the image -1 to 1

def image_postprocess(tensor):
    transform = transforms.Normalize(mean=[-0.40760392, -0.45795686, -0.48501961], 
                                     std=[1,1,1]) #다시 원래 대로 정규화해 준다.
    #transforms은 선처리를 위한 함수, transforms.Compose는 선처리를 위한 컨테이너이다.
    
    img = transform(tensor.clone()) #텐서의 값에 변화가 적용되지 않도록 텐서를 복제한다.
    img[img>1] = 1    
    img[img<0] = 0
    img = 2*(img-0.5)
    return img #이미지가 -1 에서 1 사이의 값을 가지게 한다.

# show image given a tensor as input

def imshow(tensor): #이미지 출력
    image = tensor.clone().cpu()
    image = image.view(3, image_size, image_size)
    image = transforms.ToPILImage()(image) #PIL 이미지로 재 변환
    plt.imshow(image)
    plt.show()